# Monarq Device

This document is aimed to explain what you need to know in order to use the MonarqDevice.

This device lets you communicate directly with MonarQ without using Snowflurry as an intermediate. This has the advantage of loosing the need to use Julia or other time consuming precompilings and calls. The MonarqDevice is also equipped with a compiler made by Calcul Quebec, which is optimized to work on MonarQ.


## Default usage

Here is a typical workflow for using the ```monarq.default``` device : 

0. First and foremost, you have to make sure to have python version ```3.12.x``` intalled

1. install the plugin :

In [ ]:
%pip install pennylane_snowflurry

In [ ]:
# alternatively, install the plugin from the cloned github repo : 

%git clone https://github.com/calculquebec/pennylane-snowflurry.git
%cd pennylane-snowflurry
%pip install -e .

2. import dependencies :

In [9]:
import pennylane as qml
from pennylane_snowflurry.API.api_client import MonarqClient
from dotenv import dotenv_values

3. create a device using a client

In [16]:
# this line is optional if you don't use .env files to stock your credentials
conf = dotenv_values(".env")

# change the values here for your credentials
my_client = MonarqClient(conf["HOST"], conf["USER"], conf["ACCESS_TOKEN"], conf["PROJECT_NAME"]) 
dev = qml.device("monarq.default", client=my_client, wires=[0, 1, 2], shots=1000)

4. create your circuit

In [11]:
@qml.qnode(dev)
def circuit():
    qml.Hadamard(0)
    qml.CNOT([0, 1])
    qml.CNOT([1, 2])
    
    return qml.counts()

5. run your circuit and use results as you see fit

In [12]:
results = circuit()
print(results)

{'000': 353, '001': 33, '010': 49, '011': 72, '100': 176, '101': 47, '110': 76, '111': 194}


## Client

In order to run jobs on Monarq, you will need to provide information to the device :

- host : the url to which monarq can be communicated with.
- user : your identifier
- access token : the access pass that identifies you as a rightful user
- realm : usually "calculqc"

in order to provide those informations to the device, you will need to store them in a ```ApiClient```. This is done by :

- importing the client class :

In [13]:
from pennylane_snowflurry.API.api_client import MonarqClient

- creating a client and suplying your informations to it :

In [14]:
my_client = MonarqClient("host", "user", "access_token", "realm")

- passing the client to the device :

In [15]:
dev = qml.device("monarq.default", client=my_client)

## Configuration

The default behaviour of MonarqDevice's transpiler goes as such :

1. Decompose 3+ qubits gates and non-standard gates to a subset that is easier to optimize (clifford-t gates)
2. Map the circuit's wires to the machines qubits by using the circuit's and machine's connectivity graphs
3. If any 2+ qubit gates are not connected in given mapping, connect them using swaps
4. Optimize the circuit (commute gates, merge rotations, remove inverses and trivial gates)
5. Decompose non-native gates to native ones

The transpiler uses benchmarks of the machine's accuracy in the placement and routing part.

You can change the behaviour of the transpiler by passing a MonarqConfig object to the device. Tread carefully while using this feature because it could lead to your circuit not being ran.

Here's how to do it :


1. import the config and classes that interest you:

In [17]:
from pennylane_snowflurry.transpiler.transpiler_config import MonarqDefaultConfig, TranspilerConfig

from pennylane_snowflurry.transpiler.steps.base_decomposition import CliffordTDecomposition
from pennylane_snowflurry.transpiler.steps.placement import ASTAR
from pennylane_snowflurry.transpiler.steps.routing import Swaps
from pennylane_snowflurry.transpiler.steps.optimization import IterativeCommuteAndMerge
from pennylane_snowflurry.transpiler.steps.native_decomposition import MonarqDecomposition

For simplicity, we'll store the transpiling steps in variables : 

In [18]:
base_decomp = CliffordTDecomposition()
placement = ASTAR()
routing = Swaps()
optimization = IterativeCommuteAndMerge()
native_decomp = MonarqDecomposition()

2. create a config object and set the behaviour you want

In [19]:
# this is the default configuration (it is used by default in monarq.default)
default_config = MonarqDefaultConfig()

# This is an equivalent config, built up by hand using the TranspilerConfig base class
default_config = TranspilerConfig(base_decomp, placement, routing, optimization, native_decomp)

# Say you are confident that your circuit is well placed and routed, you can skip those steps by creating a custom config, and not using placement / routing steps
custom_config = TranspilerConfig(base_decomp, optimization, native_decomp)

3. pass the config object to the device, along with the client and other arguments

In [21]:
my_client = MonarqClient("host", "user", "access_token", "realm")

dev = qml.device("monarq.default", client=my_client, behaviour_config=custom_config)

PS. You can set the placement and routing steps not to use benchmarking if you want to. 

To do so, just set ```use_benchmark``` to ```False``` in the MonarqDefaultConfig's constructor

In [22]:
my_config = MonarqDefaultConfig(False)

Alternatively, set ```use_benchmark``` to ```False``` in the placement / routing steps

In [23]:
placement_no_benchmark = ASTAR(use_benchmark = False)
routing_no_benchmark = Swaps(use_benchmark = False)

config_no_benchmark = TranspilerConfig(base_decomp, placement_no_benchmark, routing_no_benchmark, optimization, native_decomp)

## Custom transpiler steps

You can also create custom transpiler steps. There are two types of steps : 
- **preprocessing** : which are done on the quantum circuit before the execution on the quantum computer
- **postprocessing** which are done on the results after the execution on the quantum computer. 

Here's how to define custom steps : 

1. Import the base transpiler step that interests you (in our case, we are going to use preprocessing)

In [ ]:
from pennylane_snowflurry.transpiler.steps.interfaces.pre_processing import PreProcStep

2. create a class that inherits the base step, and implement the execute function as you see fit

In [ ]:
class MyCustomStep(PreProcStep):
    def execute(self, tape):
        """
        prints every operation from the tape, without altering it
        """
        for op in tape.operations:
            print(op)
        return tape

3. You can then add this step to your config by creating a custom config object

In [26]:
custom_step = MyCustomStep()

debug_config = TranspilerConfig(base_decomp, custom_step, placement, routing, optimization, native_decomp)